In [22]:
using Pipe: @pipe

mutable struct Rule
    name::String
    ranges::Array{Tuple{Int, Int}, 1}
end

mutable struct RulePos
    rule::Rule
    possible::Array{Int, 1}
end

In [23]:
function parse_rule(line)
    m = match(r"^([a-z\s]+): (\d+)-(\d+) or (\d+)-(\d+)$", line)
    return Rule(m[1], [
        (parse(Int, m[2]), parse(Int, m[3])),
        (parse(Int, m[4]), parse(Int, m[5]))
    ])
end

parse_ticket(line) = parse.(Int, split(line, ","))

parse_ticket (generic function with 1 method)

In [24]:
get_input(file) = open(file) do f
    str = read(f, String)
    parts = split(str, "\n\n")

    rules = parts[1] |> x -> split(x, "\n") |> x -> parse_rule.(x)
    your_ticket = split(parts[2], "\n")[2] |> parse_ticket
    other_tickets = split(parts[3], "\n")[2:end] |> x -> parse_ticket.(x)

    (rules, your_ticket, other_tickets)
end

get_input (generic function with 1 method)

In [25]:
rules, yours, others = get_input("16.dat");

In [26]:
rule_valid_for(rule::Rule, num::Int) =
    reduce((old, (low, high)) -> old || (low <= num && num <= high), rule.ranges; init=false)

any_valid_for(rules::Array{Rule, 1}, num::Int) = 
    reduce((old, r) -> old || rule_valid_for(r, num), rules; init=false)

ticket_valid(ticket, rules) =
    reduce((old, num) -> old && any_valid_for(rules, num), ticket; init=true)

ticket_valid (generic function with 1 method)

In [27]:
find_invalids(tickets, rules) =
    @pipe reduce(vcat, tickets) |> filter(v -> !any_valid_for(rules, v), _) |> reduce(+, _)

@show find_invalids(others, rules)

find_invalids(others, rules) = 24110


24110

In [28]:
valid_others = filter(x -> ticket_valid(x, rules), others);

In [34]:
function find_rule_pos(tickets, rules)
    poses = [RulePos(r, range(1, length(tickets[1]); step=1)) for r in rules]
    
    for ticket in tickets
        for pos in poses
            for p in pos.possible
                if !rule_valid_for(pos.rule, ticket[p])
                    pos.possible = filter(x -> x != p, pos.possible)
                end
            end
        end
    end
    
    sorted = sort(poses, by=x -> length(x.possible))
    solved = Dict{String, Int}()
    
    for s in sorted
        for p in s.possible
            if !(p in values(solved))
                solved[s.rule.name] = p
                break
            end
        end
    end
    
    solved
end

find_rule_pos (generic function with 1 method)

In [35]:
@pipe find_rule_pos(valid_others, rules) |>
    filter(v -> startswith(v[1], "departure"), _) |>
    collect |> map(v -> v[2], _) |> reduce(*, _) |> println

2904720
